# 🎯 AI Director Fine-tuning on Google Colab

## ขั้นตอนการเทรน (แบบรันทีละ cell)

**ระยะเวลา:** 3-4 ชั่วโมง (T4) หรือ 1-2 ชั่วโมง (L4)

**GPU ที่ต้องใช้:**
- Runtime → Change runtime type → T4 GPU (Free tier)
- หรือ L4 GPU (Colab Pro)

**Dataset:**
- 148 training samples
- 18 validation samples  
- 19 test samples
- รวม 185 samples จาก 8 brands

---

## 📌 Step 1: เชื่อมต่อ Google Drive

Upload folder `ai-director-colab` ไปที่ Google Drive ของคุณก่อน  
วางไว้ที่: `/content/drive/MyDrive/ai-director-colab/`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✅ เชื่อมต่อ Google Drive สำเร็จ!")

## 📌 Step 2: ตรวจสอบไฟล์ที่ Upload มา

In [ ]:
import os

# ตรวจสอบว่ามี folder ai-director-colab หรือไม่
project_path = '/content/drive/MyDrive/ai-director-colab/ai-director-colab'

if os.path.exists(project_path):
    print("✅ พบ folder ai-director-colab!")
    print("\n📁 โครงสร้างไฟล์:")
    !ls -lh {project_path}
else:
    print("❌ ไม่พบ folder ai-director-colab/ai-director-colab")
    print("กรุณา upload folder ไปที่ /content/drive/MyDrive/ai-director-colab/")

## 📌 Step 3: Copy ไฟล์มาที่ Colab (เร็วกว่า)

Copy จาก Drive มาเก็บใน Colab storage เพื่อให้การอ่านไฟล์เร็วขึ้นระหว่างเทรน

In [ ]:
!cp -r /content/drive/MyDrive/ai-director-colab/ai-director-colab /content/

print("✅ Copy ไฟล์สำเร็จ!")
print("\n📂 ไฟล์ใน /content/ai-director-colab:")
!ls -lh /content/ai-director-colab

## 📌 Step 4: ตรวจสอบ GPU

In [ ]:
!nvidia-smi

import torch

print(f"\n🎮 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("❌ ไม่พบ GPU! กรุณาเปลี่ยน Runtime เป็น GPU")
    print("Runtime → Change runtime type → T4 GPU")

## 📌 Step 5: ติดตั้ง Dependencies

ติดตั้ง libraries ที่จำเป็นสำหรับการเทรน (~5 นาที)

In [ ]:
!pip install -q transformers>=4.48.0 peft>=0.13.2 accelerate>=1.2.1 bitsandbytes>=0.45.0
!pip install -q datasets>=3.2.0 loguru>=0.7.3 tqdm>=4.67.1
!pip install -q sentencepiece>=0.2.0 protobuf>=5.29.3

print("✅ ติดตั้ง dependencies สำเร็จ!")

# ตรวจสอบ version
import transformers
import peft
print(f"\n📦 Transformers: {transformers.__version__}")
print(f"📦 PEFT: {peft.__version__}")

## 📌 Step 6: ตรวจสอบ Dataset

ดูข้อมูลที่จะใช้เทรน

In [ ]:
import json

# นับจำนวน samples
def count_jsonl_lines(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)

data_path = '/content/ai-director-colab/data'

train_count = count_jsonl_lines(f'{data_path}/train_v2.jsonl')
val_count = count_jsonl_lines(f'{data_path}/val_v2.jsonl')
test_count = count_jsonl_lines(f'{data_path}/test_v2.jsonl')

print("📊 Dataset Statistics:")
print(f"  - Training:   {train_count} samples")
print(f"  - Validation: {val_count} samples")
print(f"  - Test:       {test_count} samples")
print(f"  - Total:      {train_count + val_count + test_count} samples")

# แสดงตัวอย่างข้อมูล
print("\n📝 ตัวอย่างข้อมูล (sample 1):")
with open(f'{data_path}/train_v2.jsonl', 'r', encoding='utf-8') as f:
    sample = json.loads(f.readline())
    print(f"Instruction: {sample['instruction']}")
    print(f"\nInput: {sample['input'][:200]}...")
    print(f"\nOutput: {sample['output'][:200]}...")
    print(f"\nMetadata: {sample['metadata']}")

## 📌 Step 7: เริ่มเทรน! 🚀

**ใช้เวลา:** 3-4 ชั่วโมง (T4) หรือ 1-2 ชั่วโมง (L4)

**จะเห็น:**
- Training progress bar
- Loss ลดลงเรื่อยๆ
- Evaluation results ทุก 50 steps

**Output:**
- LoRA adapters → `models/qwen-7b-ai-director-v2/`
- Training logs → `logs/`
- Checkpoints ทุก 50 steps

In [ ]:
# เปลี่ยนไปที่ scripts directory
%cd /content/ai-director-colab/scripts

# รันการเทรน
!python finetune_lora.py

## 📌 Step 8: ตรวจสอบผลลัพธ์

In [ ]:
import json
import os

# ตรวจสอบว่าเทรนเสร็จหรือยัง
model_path = '/content/ai-director-colab/scripts/models/qwen-7b-ai-director-v2'

if os.path.exists(model_path):
    print("✅ เทรนเสร็จแล้ว! พบโมเดล:")
    !ls -lh {model_path}
    
    # อ่านผลการ evaluate
    eval_file = f'{model_path}/eval_results.json'
    if os.path.exists(eval_file):
        with open(eval_file, 'r') as f:
            results = json.load(f)
        print("\n📊 Evaluation Results:")
        for key, value in results.items():
            print(f"  {key}: {value}")
else:
    print("⏳ กำลังเทรน... โปรดรอ")

## 📌 Step 9: ทดสอบโมเดล (Inference)

ลองใช้โมเดลที่เทรนเสร็จแล้ว

In [ ]:
# ตรวจสอบและ extract โมเดลจากไฟล์ zip
import os
import zipfile

base_path = "/content/drive/MyDrive/ai-director-colab"
model_folder = f"{base_path}/trained_models/qwen-7b-ai-director-v2"

# ค้นหาไฟล์ zip
print("🔍 ค้นหาไฟล์ zip ที่เกี่ยวข้อง...\n")
!ls -lh {base_path}/trained_models*.zip 2>/dev/null || echo "ไม่พบไฟล์ zip"

# ถ้ายังไม่มีโฟลเดอร์โมเดล ให้ extract
if not os.path.exists(model_folder):
    # หาไฟล์ zip ที่มีคำว่า trained_models
    zip_files = !ls {base_path}/trained_models*.zip 2>/dev/null
    
    if zip_files and zip_files[0]:
        zip_path = zip_files[0]
        print(f"📦 พบไฟล์ zip: {zip_path}")
        print("🔄 กำลัง extract... (ใช้เวลา 1-2 นาที)")
        
        # Extract ไปที่ base_path
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(base_path)
        
        print("✅ Extract สำเร็จ!")
    else:
        print("❌ ไม่พบไฟล์ zip และไม่มีโฟลเดอร์โมเดล")
        print("📌 คุณต้อง:")
        print("   1. Copy โฟลเดอร์ trained_models จาก account เก่า")
        print("   2. หรือ upload ไฟล์ zip ที่ download มา")
else:
    print(f"✅ พบโฟลเดอร์โมเดลแล้ว: {model_folder}")

# ตรวจสอบไฟล์
print(f"\n🔍 ตรวจสอบไฟล์ใน: {model_folder}\n")
if os.path.exists(model_folder):
    !ls -lh {model_folder}
    
    print("\n📋 ไฟล์ที่จำเป็น:")
    required_files = ["adapter_config.json", "adapter_model.safetensors"]
    for file in required_files:
        exists = os.path.exists(f"{model_folder}/{file}")
        status = "✅" if exists else "❌"
        print(f"  {status} {file}")
else:
    print("❌ ยังไม่พบโฟลเดอร์")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

# โหลดโมเดล
print("🔄 กำลังโหลดโมเดล...")

base_model = "Qwen/Qwen2.5-7B-Instruct"
lora_path = "/content/drive/MyDrive/ai-director-colab/trained_models/qwen-7b-ai-director-v2"

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

# ตั้งค่า 4-bit quantization (เหมือนตอนเทรน)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# โหลด base model พร้อม 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# โหลด LoRA adapters
model = PeftModel.from_pretrained(model, lora_path)
model.eval()

print("✅ โหลดโมเดลสำเร็จ!")

In [ ]:
# ฟังก์ชันสำหรับ generate
def generate_content(instruction, input_text, max_length=512):
    prompt = f"""<|im_start|>system
You are an AI Director for marketing content creation. Generate on-brand content based on the instructions.<|im_end|>
<|im_start|>user
{instruction}

{input_text}<|im_end|>
<|im_start|>assistant
"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    # Decode เฉพาะ generated tokens
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # ตัดเอาเฉพาะส่วน assistant response
    # ตัด prompt ทั้งหมดออก แล้วเอาเฉพาะส่วนที่ต่อจาก "assistant"
    if "assistant" in full_response:
        parts = full_response.split("assistant")
        # เอาส่วนสุดท้ายหลังจาก "assistant" ที่เป็นคำตอบจริงๆ
        result = parts[-1].strip()
        return result
    
    return full_response.strip()

print("✅ พร้อมใช้งาน!")

### ทดสอบ 1: เขียน Instagram Caption

In [ ]:
instruction = "เขียน Instagram caption สำหรับ CoffeeLab"
input_text = """Brand: CoffeeLab
Product: Single Origin Ethiopia
Tone: warm, friendly, expert
Context: New product launch"""

print("📝 Instruction:", instruction)
print("\n💬 Input:", input_text)
print("\n🤖 AI Director Output:")
print("=" * 50)
print(generate_content(instruction, input_text))

### ทดสอบ 2: สร้าง Visual Prompt

In [ ]:
instruction = "สร้าง Midjourney prompt สำหรับ TechZone"
input_text = """Brand: TechZone
Context: Product showcase - Gaming Keyboard
Visual Style: dark cyberpunk, neon RGB lighting
Aspect Ratio: 16:9"""

print("📝 Instruction:", instruction)
print("\n💬 Input:", input_text)
print("\n🤖 AI Director Output:")
print("=" * 50)
print(generate_content(instruction, input_text))

### ทดสอบ 3: ตอบ Customer Review

In [ ]:
instruction = "ตอบกลับความคิดเห็นของลูกค้า"
input_text = """Brand: CoffeeLab
Tone: friendly, warm
Comment: รสชาติเปรี้ยวมาก ไม่อร่อยเลย
Sentiment: negative"""

print("📝 Instruction:", instruction)
print("\n💬 Input:", input_text)
print("\n🤖 AI Director Output:")
print("=" * 50)
print(generate_content(instruction, input_text))

## 📌 Step 10: บันทึกโมเดลกลับไปที่ Google Drive

Copy โมเดลที่เทรนเสร็จกลับไป Drive เพื่อใช้งานต่อ

In [ ]:
# Copy โมเดล + logs กลับไป Drive
!cp -r /content/ai-director-colab/scripts/models /content/drive/MyDrive/ai-director-colab/ai-director-colab/
!cp -r /content/ai-director-colab/scripts/logs /content/drive/MyDrive/ai-director-colab/ai-director-colab/

print("✅ บันทึกโมเดลไปที่ Google Drive สำเร็จ!")
print("\n📂 Location:")
print("  - /content/drive/MyDrive/ai-director-colab/ai-director-colab/models/")
print("  - /content/drive/MyDrive/ai-director-colab/ai-director-colab/logs/")

## 🎉 เสร็จสิ้น!

### สิ่งที่คุณได้:
- ✅ LoRA adapters ที่เทรนเสร็จแล้ว
- ✅ Training logs
- ✅ Evaluation results
- ✅ โมเดลพร้อมใช้งาน

### Next Steps:
1. ใช้ `inference_rag.py` สำหรับ production
2. เพิ่มแบรนด์ใหม่ใน `brands_v2.json` (ไม่ต้อง retrain!)
3. Fine-tune ต่อด้วยข้อมูลเพิ่ม

### 📚 Resources:
- Full documentation: `/docs/README_V2.md`
- Course material: `course_ai-assistant_v3.4.md`

---

**Version:** 2.0  
**Date:** January 4, 2026